# 기초 사용법

[opencv document](https://docs.opencv.org/4.5.5/)

In [1]:
import cv2

In [ ]:
cv2.__version__

In [ ]:
img = cv2.imread('./data/lenna.bmp')

print(img.shape)

cv2.namedWindow("image")
cv2.imshow("image", img)

## 종료조건
cv2.waitKey()
cv2.destroyAllWindows()

### Workshop : 영상 파일 읽기 및 화면 표시

1. './data/lena.jpg' 파일을 읽어 들여서 컬러로 화면에 표시하고 키보드에서 아무 키나 누르면 윈도우가 사라지도록 하라.

2. './data/lena.jpg' 파일을 읽어 들여서 흑백으로 화면에 표시하고 키보드에서 아무 키나 누르면 윈도우가 사라지도록 하라.

In [ ]:
# TODO
imgFile = './data/lena.jpg'

imgColor = cv2.imread(imgFile, cv2.IMREAD_COLOR)
imgGray = cv2.imread(imgFile, cv2.IMREAD_GRAYSCALE)

cv2.imshow("Lene Color", imgColor)
cv2.imshow("Lene GrayScale", imgGray)

cv2.waitKey()
cv2.destroyAllWindows()


### Workshop : 영상 파일 저장

1. './data/lena.jpg' 파일을 읽어 들여서 bmp 파일 형식의 './out/Lena.bmp'로 저장하라.

2. './data/lena.jpg' 파일을 읽어 들여서 png 파일 형식의 './out/Lena.png'로 저장하라.

3. './data/lena.jpg' 파일을 읽어 들여서 png 파일 형식의 './out/Lena2.png'로 저장하되 압축률 90%를 적용하라. 

4. './data/lena.jpg' 파일을 읽어 들여서 jpg 파일 형식의 './out/Lena2.jpg'로 저장하되 압축률 50%를 적용하라 

In [ ]:
# TODO
imgFile = './data/lena.jpg'
imgColor = cv2.imread(imgFile, cv2.IMREAD_COLOR)

cv2.imwrite('./out/Lena.bmp', imgColor)
cv2.imwrite('./out/Lena.png', imgColor)

cv2.imwrite('./out/Lena2.png', imgColor, [cv2.IMWRITE_PNG_COMPRESSION, 9]) #0~9
cv2.imwrite('./out/Lena2.jpg', imgColor, [cv2.IMWRITE_JPEG_QUALITY, 90]) #0~100

### Workshop : matplotlib 1 컬러영상 표시
1. openCV를 이용해 './data/lena.jpg' 파일을 읽어 들인 후 matplotlib.pyplot의 imshow() 함수를 사용해 화면에 표시하라. 
- 결과 확인
2. openCV를 이용해 './data/lena.jpg' 파일을 읽어 들인 후 matplotlib.pyplot의 imshow() 함수를 사용해 화면에 표시하라. 
- 표시된 결과가 원본과 동일한 색조가 되도록 변환 후 화면에 표시
3. openCV를 이용해 './data/lena.jpg' 파일을 읽어 들인 후 matplotlib.pyplot의 imshow() 함수를 사용해 화면에 표시하라. 
- numpy ndarray의 색인 문법으로 컬러 채널의 순서를 변경

In [ ]:
#TODO
import matplotlib.pyplot as plt
imgFile = './data/lena.jpg'
img = cv2.imread(imgFile)

plt.imshow(img)

In [ ]:
imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(imgRGB)

In [ ]:
img.shape

In [ ]:
import numpy as np

imgRGB = np.zeros(img.shape, dtype='uint8')
print(imgRGB.dtype)

imgRGB[:, :, 2] = img[:, :, 0] #blue channel
imgRGB[:, :, 1] = img[:, :, 1] #green channel
imgRGB[:, :, 0] = img[:, :, 2] #red channel

plt.imshow(imgRGB)

In [ ]:
import numpy as np

imgRGB = np.zeros(img.shape)
print(imgRGB.dtype)

imgRGB[:, :, 2] = img[:, :, 0] #blue channel
imgRGB[:, :, 1] = img[:, :, 1] #green channel
imgRGB[:, :, 0] = img[:, :, 2] #red channel

imgRGB= imgRGB/255.0 #0~1

plt.imshow(imgRGB)

# 행렬 연산

### 행렬 : numpy ndarray

In [ ]:
import sys

img = cv2.imread('./data/cat.bmp', cv2.IMREAD_GRAYSCALE)

if img is None:
    print("Image load failed!!!")
    sys.exit()
    
print("type(img) : ", type(img))
print("shape(img) : ", img.shape)

cv2.imshow("img", img)
cv2.waitKey()
cv2.destroyAllWindows()

### 행렬의 생성과 초기화

In [ ]:
img1 = np.empty((480, 640), np.uint8)
img2 = np.zeros((480, 640), np.uint8)
img3 = np.ones((480, 640), np.uint8)

img4 = np.full((480, 640), 0, np.float32)
print(type(img1), type(img2), type(img3), type(img4))

In [ ]:
l = [[1, 2, 3, 4],
     [5, 6, 7, 8],
     [9, 10, 11, 12]] #python list

mat = np.array(l)
print(type(mat), '\n', mat)

In [ ]:
mat[0, 1] = 100
mat

In [ ]:
mat[2, :] = 200
mat

### 행렬의 복사

In [ ]:
img1 = cv2.imread('./data/cat.bmp')

img2 = img1
img3 = img1.copy()

img1[:, :] = (0, 255, 255) #yellow

cv2.imshow("img2", img2)
cv2.imshow("img3", img3)
cv2.waitKey()
cv2.destroyAllWindows()

### 행렬 원소값 참조

In [ ]:
np.arange(12).reshape(3, 4)

In [ ]:
mat1 = np.array(np.arange(12).reshape(3, 4))

mat2 = np.zeros(mat1.shape, type(mat1))

h, w = mat1.shape

for j in range(h):
    for i in range(w):
        mat2[j, i] = mat[j, i] + 10
        
print(mat1)
print(mat2)

In [ ]:
mat2 = mat1 + 10 # broadcasting
mat2

### 행렬 연산

In [ ]:
mat1 = np.ones((3, 4), np.int32)
mat2 = np.arange(12).reshape(3, 4)
mat3 = mat1 + mat2
mat4 = mat2 * 2

### Workshop : 이미지에 테두리 그리기

In [ ]:
img = plt.imread('./data/psj.png')
plt.imshow(img)

In [ ]:
img.shape, type(img)

In [ ]:
img[0, 0]

In [ ]:
img[0, 0, 0] = 1 #R channel에 최대 밝기값 부여
img[0, 0, 1] = 0 #G channel에 최소 밝기값 부여
img[0, 0, 2] = 0 #B channel에 최소 밝기값 부여

In [ ]:
img[0, 0]

In [ ]:
plt.imshow(img)

In [ ]:
# 좌상단 끝(10*10)을 빨가색으로 바꿔보기
img = plt.imread('./data/psj.png')
img[0:10, 0:10, 0] = 1
img[0:10, 0:10, 1] = 0
img[0:10, 0:10, 2] = 0
plt.imshow(img)

In [ ]:
img = plt.imread('./data/psj.png')
img[0:150, :, 0] = 1
img[0:150, :, 1] = 1
img[0:150, :, 2] = 1
plt.imshow(img)

In [ ]:
img.shape[1] - 10

In [ ]:
img.shape[0]

In [ ]:
img = plt.imread('./data/psj.png')
# 상단 가로
img[0:10, :] = [1, 1, 0, 1]
# 좌측 세로
img[:, 0:10] = [1, 1, 0, 1]
# 우측 세로
img[:, img.shape[1] - 10:] = [1, 1, 0, 1]
# 하단 가로
img[img.shape[0]-10:, :] = [1, 1, 0, 1]

plt.imshow(img)

In [ ]:
# 반전
img = plt.imread('./data/psj.png')
img[:, :, :3] = 1-img[:, :, :3]
plt.imshow(img)

In [ ]:
# 4번째 채널(alpha channel)
img = plt.imread('./data/psj.png')
img[:, :, 3] = 0.3
plt.imshow(img)

# 동영상 파일 다루기

### 카메라 입력

In [ ]:
cap = cv2.VideoCapture(0) # 카메라 디바이스, 동영상 파일명, 스트리밍 주소

if not cap.isOpened():
    print("Camera open failed!!")
    sys.exit()


w, h = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
print("frame width, height:", w, h)

while True:
    ret, frame = cap.read()
    
    if not ret:
        break
    
    inversed = ~frame
    
    cv2.imshow("frame", frame)
    cv2.imshow("inversed", inversed)
    
    if cv2.waitKey(10) == 27: #sleep 효과
        break

if cap.isOpened():
    cap.release()

cv2.destroyAllWindows()

### 동영상 파일

In [2]:
cap = cv2.VideoCapture('./data/stopwatch.avi') # 카메라 디바이스, 동영상 파일명, 스트리밍 주소

if not cap.isOpened():
    print("Camera open failed!!")
    sys.exit()


w, h = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

print("frame width, height:", w, h)
print("fps : ", fps)

delay = round(1000/fps)

while True:
    ret, frame = cap.read()
    
    if not ret:
        break;
        
    inversed = ~frame
    
    cv2.imshow("frame", frame)
    cv2.imshow("inversed", inversed)
    
    if cv2.waitKey(delay) == 27: #sleep 효과
        break

if cap.isOpened():
    cap.release()

cv2.destroyAllWindows()

frame width, height: 640 480
fps :  30.0


### 동영상 저장

In [4]:
cap = cv2.VideoCapture(0) # 카메라 디바이스, 동영상 파일명, 스트리밍 주소

if not cap.isOpened():
    print("Camera open failed!!")
    sys.exit()


w, h = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
print("frame width, height:", w, h)
print("fps : ", fps)


fourcc = cv2.VideoWriter_fourcc(*"DIVX")

outputVideo = cv2.VideoWriter('./out/output2.avi', fourcc, fps, (w, h))

while True:
    ret, frame = cap.read()
    
    if not ret:
        break;d
    
    inversed = ~frame
    
    outputVideo.write(inversed)
    
    cv2.imshow("frame", frame)
    cv2.imshow("inversed", inversed)
    
    if cv2.waitKey(10) == 27: #sleep 효과
        break

if cap.isOpened():
    cap.release()
    
outputVideo.release()
cv2.destroyAllWindows()

frame width, height: 640 480
fps :  30.0
